In [ ]:
#check requirement.txt file for correct package versioning. You will very likely run into issues if you have discrepant package versions
#see also: http://ski.clps.brown.edu/hddm_docs/tutorial.html for correct set up. I used Conda to create the virtual environment for this project.
#I ran into many issues trying to get this project to run on M1 macbook and reccomend you do not attempt
#Warning, this script takes a long time to run! You can reduce the # of samples to speed up the process, although this will have other consequences that might not be ideal

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import hddm
import numpy as np
from patsy import dmatrix  # for generation of (regression) design matrices
import os
import earthpy as et #some helpful functions for path nav
import seaborn as sns
from pandas import Series  # to manipulate data-frames generated by hddm

FileNotFoundError: [Errno 2] No such file or directory: '/data/cleaned/ddm_data.csv'

In [ ]:
#make sure to save the project in home directory and then run below code to set correct directory for project
os.chdir(os.path.join(et.io.HOME, 'Computational_Mech_manuscript'))
os.getcwd()
samples = 20000 #of samples for MCMC
data = hddm.load_csv('./data/cleaned/ddm_data.csv')

In [ ]:
#link function if needed
def z_link_func(x, data=data):
    stim = (np.asarray(dmatrix('0 + C(s, [[0], [1]])',
                              {'s': data.condition.loc[x.index]},return_type='dataframe'))
    )
    # Apply z = (1 - x) to flip them along 0.5
    z_flip = np.subtract(stim, x.to_frame())
    # The above inverts those values we do not want to flip,
    # so invert them back
    z_flip[stim == 0] *= -1
    return z_flip

In [ ]:
#custom ppc function generating 500 simulated data sets the size equal to #of trials and participants
import pymc as pm
import numpy as np
import pymc.progressbar as pbar
def _parents_to_random_posterior_sample(bottom_node, pos=None):
    """Walks through parents and sets them to pos sample."""
    for i, parent in enumerate(bottom_node.extended_parents):
        if not isinstance(parent, pm.Node): # Skip non-stochastic nodes
            continue

        if pos is None:
            # Set to random posterior position
            pos = np.random.randint(0, len(parent.trace()))

        assert len(parent.trace()) >= pos, "pos larger than posterior sample size"
        parent.value = parent.trace()[pos]
def _post_pred_generate(bottom_node, samples=500, data=None, append_data=True):
    """Generate posterior predictive data from a single observed node."""
    datasets = []
    ##############################
    # Sample and generate stats
    for sample in range(samples):
        _parents_to_random_posterior_sample(bottom_node)
        # Generate data from bottom node
        sampled_data = bottom_node.random()
        if append_data and data is not None:
            sampled_data.reset_index(inplace=True)  # Only modification of original Kabuki code
            sampled_data = sampled_data.join(data.reset_index(), lsuffix='_sampled')
        datasets.append(sampled_data)
    return datasets
def post_pred_gen(model, groupby=None, samples=500, append_data=False, progress_bar=True):
    results = {}

    # Progress bar
    if progress_bar:
        n_iter = len(model.get_observeds())
        bar = pbar.progress_bar(n_iter)
        bar_iter = 0
    else:
        print("Sampling...")

    if groupby is None:
      iter_data = ((name, model.data.iloc[obs['node'].value.index]) for name, obs in model.iter_observeds())

    else:
        iter_data = model.data.groupby(groupby)

    for name, data in iter_data:
        node = model.get_data_nodes(data.index)

        if progress_bar:
            bar_iter += 1
            bar.update(bar_iter)

        if node is None or not hasattr(node, 'random'):
            continue # Skip

        ##############################
        # Sample and generate stats
        datasets = _post_pred_generate(node, samples=samples, data=data, append_data=append_data)
        results[name] = pd.concat(datasets, names=['sample'], keys=list(range(len(datasets))))

    if progress_bar:
        bar_iter += 1
        bar.update(bar_iter)

    return pd.concat(results, names=['node'])

In [ ]:
# Z & V model: Bias in both the starting point and drift rate

In [ ]:
#Bias in both starting point and drift rate
v_reg = {'model': 'v ~ 1 + condition + stim_n + stim_weight',  'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 ',  'link_func': lambda x: x}
reg_descr = [v_reg, z_reg]
FullModel = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)
FullModel.find_starting_values()
FullModel.sample(samples, burn=samples/10, thin=2, dbname='FullModel_traces.db', db='pickle')

In [ ]:
#check convergence using gelman-rubin
#this model uses the reg_descr defined above, be sure to run in order
from kabuki.analyze import gelman_rubin
models = []
for i in range(5):
    m = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)
    m.find_starting_values()
    m.sample(5000, burn=2)
    models.append(m)
gelman_rubin(models)

In [ ]:
#check posterior convergence via chains
FullModel.plot_posteriors()

In [ ]:
#plot posteriors. This appears in SOM
z_int,v_int, v_cond =  FullModel.nodes_db.loc[[
                                              "z_Intercept",
                                              "v_Intercept", "v_condition"], 'node']
hddm.analyze.plot_posterior_nodes([z_int])
plt.xlabel('Starting point')
plt.ylabel('Posterior probability')
plt.title('posteriors of within-subject starting point effects.')
hddm.analyze.plot_posterior_nodes([v_int])
plt.xlabel('Drift-rate when outgroup is better')
plt.ylabel('Posterior probability')
plt.title('posteriors of within-subject drift-rate effects.')
hddm.analyze.plot_posterior_nodes([v_cond])
plt.xlabel('Drift-rate when ingroup is better')
plt.ylabel('Posterior probability')
plt.title('posteriors of within-subject drift-rate effects.')

In [ ]:
#this chunk is for saving the model outputs (e.g., traces). Uncomment to run
#save subj parms
# stats = FullModel.gen_stats()
# stats.to_csv('saved_hddm_models_and_parms/model_outputs/subject_parms/')
# #save model traces
# a, t = FullModel.nodes_db.node[['a', 't']]
# z_0, v_0,v_m,v_s_n, v_s_w = FullModel.nodes_db.node[['z_Intercept','v_Intercept', 'v_condition', 'v_stim_n', 'v_stim_weight']]
# allParms = a.trace()
# allParms = np.column_stack([allParms,t.trace()])
# allParms = np.column_stack([allParms,z_0.trace()])
# allParms = np.column_stack([allParms,v_0.trace(),v_m.trace(),v_s_n.trace(),v_s_w.trace()])
# np.savetxt('saved_hddm_models_and_parms/model_outputs/trace_processed/', allParms , delimiter=",")

In [ ]:
#Bias only in starting point but not drift rate

In [ ]:
#fit bias starting point model as indicated with bias=T function
v_reg = {'model': 'v ~ 1 + stim_n + stim_weight',  'link_func': lambda x: x}
#z_reg = {'model': 'z ~ 1 ',  'link_func': lambda x: x}
reg_descr = [v_reg]
SimpleBias = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)
SimpleBias.find_starting_values()
SimpleBias.sample(samples, burn=samples/10, thin=2, dbname='simpleBias_traces.db', db='pickle')

In [ ]:
#check convergence
#this model takes in the reg_descr defined above so be sure to run in order
from kabuki.analyze import gelman_rubin
models = []
for i in range(5):
    m = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)
    m.find_starting_values()
    m.sample(5000, burn=2)
    models.append(m)

gelman_rubin(models)

In [ ]:
#check convergence via chains, the SD is often less stable.
SimpleBias.plot_posteriors()
#plot posteriors
z_int= SimpleBias.nodes_db.loc[["z_Intercept"], 'node']
hddm.analyze.plot_posterior_nodes([z_int])
plt.xlabel('Starting point')
plt.ylabel('Posterior probability')
plt.title('posteriors of within-subject starting point effects.')

In [ ]:
#save subj parms
# stats = simpleBias.gen_stats()
# stats.to_csv('saved_hddm_models_and_parms/model_outputs/subject_parms')

#Save model traces
# a, t = SimpleBias.nodes_db.node[['a', 't']]
# z_0 = SimpleBias.nodes_db.node[[ 'z_Intercept']]
# v_s_n,v_s_w,v_0 = SimpleBias.nodes_db.node[['v_stim_n','v_stim_weight','v_Intercept']]
# allParms = a.trace()
# allParms = np.column_stack([allParms,t.trace()])
# allParms = np.column_stack([allParms,z_0.trace()])
# allParms = np.column_stack([allParms,v_s_n.trace(),v_s_w.trace(),v_0.trace()])
# np.savetxt('saved_hddm_models_and_parms/model_outputs/trace_processed', allParms , delimiter=",")

In [ ]:
#Bias only in drift rate but not starting point

In [ ]:
#no bias in starting point is indicated with bias=F
v_reg = {'model': 'v ~ 1 + condition+ stim_n + stim_weight',  'link_func': lambda x: x}
reg_descr = [v_reg]
BiasDrift = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=False, p_outlier=0.05,
                                       group_only_regressors=False)
BiasDrift.find_starting_values()
BiasDrift.sample(samples, burn=samples/10, thin=2, dbname='BiasDrift_traces.db', db='pickle')

In [ ]:
#check convergence
from kabuki.analyze import gelman_rubin
models = []
for i in range(5):
    m = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)
    m.find_starting_values()
    m.sample(5000, burn=2)
    models.append(m)
gelman_rubin(models)

In [ ]:
#check posterior convergence via chains
BiasDrift.plot_posteriors()
#plot posteriors
v_int, v_cond =  BiasDrift.nodes_db.loc[["v_Intercept", "v_condition"], 'node']
hddm.analyze.plot_posterior_nodes([v_int])
plt.xlabel('Drift-rate when outgroup is better')
plt.ylabel('Posterior probability')
plt.title('posteriors of within-subject drift-rate effects.')
hddm.analyze.plot_posterior_nodes([v_cond])
plt.xlabel('Drift-rate when ingroup is better')
plt.ylabel('Posterior probability')
plt.title('posteriors of within-subject drift-rate effects.')

In [ ]:
#save subj parms
# stats = BiasDrift.gen_stats()
# stats.to_csv('saved_hddm_models_and_parms/model_outputs/subject_parms')
# # #Save model traces
# a, t = BiasDrift.nodes_db.node[['a', 't']]
# z_0,v_m  = BiasDrift.nodes_db.node[['v_Intercept', 'v_condition']]
# v_s_n,v_s_w = BiasDrift.nodes_db.node[['v_stim_n','v_stim_weight']]
# allParms = a.trace()
# allParms = np.column_stack([allParms,t.trace()])
# allParms = np.column_stack([allParms,z_0.trace(),v_m.trace()])
# allParms = np.column_stack([allParms,v_s_n.trace(),v_s_w.trace()])
# np.savetxt('saved_hddm_models_and_parms/model_outputs/trace_processed', allParms , delimiter=",")

In [ ]:
#Null Model

In [ ]:
#fit model where starting point is fixed and no difference in drift rate based on condition
v_reg = {'model': 'v ~ 1 + stim_n + stim_weight', 'link_func': lambda x: x}
reg_descr = [v_reg]
NullModel= hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=False, p_outlier=0.05,
                                       group_only_regressors=False)
NullModel.find_starting_values()
NullModel.sample(samples, burn=samples/10, thin=2, dbname='NullModel_traces.db', db='pickle')

In [ ]:
#check convergence
from kabuki.analyze import gelman_rubin
models = []
for i in range(5):
    m = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)
    m.find_starting_values()
    m.sample(5000, burn=2)
    models.append(m)
gelman_rubin(models)

In [ ]:
#check posterior convergence via chains
NullModel.plot_posteriors()

In [ ]:
#DIC model fit indices

In [ ]:
#print model fit DIC
print("full Model DIC: %f" % FullModel.dic)
print("full Model DIC: %f" % SimpleBias.dic)
print("full Model DIC: %f" % BiasDrift.dic)
print("full Model DIC: %f" % NullModel.dic)

In [ ]:
#Run Posterior predictive checks


In [ ]:
##generate posterior data for each model
PPC_FullModel = post_pred_gen(FullModel, append_data=True)
PPC_SimpleBias = post_pred_gen(SimpleBias, append_data=True)
PPC_BiasDrift = post_pred_gen(BiasDrift, append_data=True)
PPC_NullModel = post_pred_gen(NullModel, append_data=True)


In [ ]:
##compare each model's simulated data to OG data
ppc_compare_Full = hddm.utils.post_pred_stats(data, PPC_FullModel)
ppc_compare_Simple = hddm.utils.post_pred_stats(data, PPC_SimpleBias)
ppc_compare_Drift = hddm.utils.post_pred_stats(data, PPC_BiasDrift)
ppc_compare_Null = hddm.utils.post_pred_stats(data, PPC_NullModel)

In [ ]:
##print PPC
print(ppc_compare_Full)
print(ppc_compare_Simple)
print(ppc_compare_Drift)
print(ppc_compare_Null)

In [ ]:
#Parameter recovery starts here

In [ ]:
#Best fitting model (z & v)

In [ ]:
#load in data simulated from best fitting model
sim_data_sim_full = hddm.load_csv('saved_hddm_models_and_parms/param_recovery_input/FullModel_simulated.csv')
v_reg = {'model': 'v ~ 1 + condition + stim_n + stim_weight',  'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 ',  'link_func': lambda x: x}

reg_descr = [v_reg, z_reg]
FullModel_recovery = hddm.models.HDDMRegressor(sim_data_sim_full, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)
FullModel_recovery.find_starting_values()
FullModel_recovery.sample(samples, burn=samples/10, thin=2, dbname='FullModel_recovered_traces.db', db='pickle')

In [ ]:
#save sub parms from best fitting recovered model
# stats = FullModel_recovery.gen_stats()
# stats.to_csv('saved_hddm_models_and_parms/param_recovery_output')

In [ ]:
#Bias starting point model

In [ ]:
#load in data simulated from biased starting point model
sim_data_Bias = hddm.load_csv('saved_hddm_models_and_parms/param_recovery_input/SimpleBias_simulated.csv')
v_reg = {'model': 'v ~ 1 + stim_n + stim_weight',  'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 ',  'link_func': lambda x: x}
reg_descr = [v_reg, z_reg]
simpleBias_recovery = hddm.models.HDDMRegressor(sim_data_Bias, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)
simpleBias_recovery.find_starting_values()
simpleBias_recovery.sample(samples, burn=samples/10, thin=2, dbname='simpleBias_recovered_traces.db', db='pickle')

In [ ]:
#save sub parms from Bias drift model
# stats = simpleBias_recovery.gen_stats()
# stats.to_csv('saved_hddm_models_and_parms/param_recovery_output')

In [ ]:
#Bias drift rate model

In [ ]:
#load in data simulated from biased drift model
sim_data_Drift = hddm.load_csv('saved_hddm_models_and_parms/param_recovery_input/BiasDrift_simulated.csv')
v_reg = {'model': 'v ~ 1 + condition+ stim_n + stim_weight',  'link_func': lambda x: x}
reg_descr = [v_reg]
BiasDrift_recovery = hddm.models.HDDMRegressor(sim_data_Drift, reg_descr,
                                       bias=False, p_outlier=0.05,
                                       group_only_regressors=False)
BiasDrift_recovery.find_starting_values()
BiasDrift_recovery.sample(samples, burn=samples/10, thin=2, dbname='BiasDrift_recovered_traces.db', db='pickle')

In [ ]:
#save sub parms from Bias drift model
# stats = BiasDrift_recovery.gen_stats()
# stats.to_csv('saved_hddm_models_and_parms/param_recovery_output')

In [ ]:
#see R script for correlations between recovered and estimates parameters




